In [1]:
#Import required modules
import requests
import pandas as pd
import pycountry
from sqlalchemy import *

In [3]:
#Assign the URL for extracting obesity data and malnutrition data of adult and children
url_ob_adlt = "https://ghoapi.azureedge.net/api/NCD_BMI_30C"
url_ob_chld = "https://ghoapi.azureedge.net/api/NCD_BMI_PLUS2C"
url_mal_adlt = "https://ghoapi.azureedge.net/api/NCD_BMI_18C"
url_mal_chld = "https://ghoapi.azureedge.net/api/NCD_BMI_MINUS2C"

In [5]:
#Send request and receive the response
response_ob_adl = requests.get(url_ob_adlt)
response_ob_chl = requests.get(url_ob_chld)
response_mal_adl = requests.get(url_mal_adlt)
response_mal_chl = requests.get(url_mal_chld)

In [7]:
#Convert the response to a json format
data_ob_ad = response_ob_adl.json()
data_ob_ch = response_ob_chl.json()
data_ml_ad = response_mal_adl.json()
data_ml_ch = response_mal_chl.json()

In [9]:
#The data contains two keys ['@odata.context', 'value']. Extract the value details and assign it as a dataframe
df_ob_ad = pd.DataFrame(data_ob_ad['value'])
df_ob_ch = pd.DataFrame(data_ob_ch['value'])
df_ml_ad = pd.DataFrame(data_ml_ad['value'])
df_ml_ch = pd.DataFrame(data_ml_ch['value'])

In [11]:
#Add a new column for categorising the age group
df_ob_ad['Age_group'] = "Adult"
df_ob_ch['Age_group'] = "Child"
df_ml_ad['Age_group'] = "Adult"
df_ml_ch['Age_group'] = "Child"

In [13]:
#Combine adult, children data of obesity into a single dataframe and adult, children data of malnutrition into another dataframe
df_obesity = pd.concat([df_ob_ad,df_ob_ch],ignore_index=True)
df_malnutrition = pd.concat([df_ml_ad,df_ml_ch],ignore_index=True)

In [15]:
#Filter the data for the year 2012 to 2022
df_obesity = df_obesity[(df_obesity['TimeDim']>=2012) & (df_obesity['TimeDim']<=2022)].reset_index(drop=True)
df_malnutrition = df_malnutrition[(df_malnutrition['TimeDim']>=2012) & (df_malnutrition['TimeDim']<=2022)].reset_index(drop=True)

In [17]:
#Retain only required columns and rename them for legibility
req_col = ['ParentLocation',
           'Dim1',
           'TimeDim',
           'Low',
           'High',
           'NumericValue',
           'SpatialDim',
           'Age_group']
col_rename ={'ParentLocation':'Region',
             'Dim1':'Gender',
             'TimeDim':'Year',
             'Low':'LowerBound',
             'High':'UpperBound',
             'NumericValue':'Mean_Estimate',
             'SpatialDim':'Country'}
#To retain required columns
df_obesity = df_obesity[req_col]
df_malnutrition = df_malnutrition[req_col]
#To rename columns
df_obesity = df_obesity.rename(columns=col_rename)
df_malnutrition = df_malnutrition.rename(columns=col_rename)

In [19]:
#Convert the Gender column values to Male,Female,Both for legibility
df_obesity['Gender'] = df_obesity['Gender'].apply([lambda x:"Male" if x == "SEX_MLE" else "Female" if x == "SEX_FMLE" else "Both"])
df_malnutrition['Gender'] = df_malnutrition['Gender'].apply([lambda x:"Male" if x == "SEX_MLE" else "Female" if x == "SEX_FMLE" else "Both"])

In [21]:
#Create a new column named CI_Width with the formula CI_Width = High - Low for obesity and malnutrition data
df_obesity['CI_width'] = df_obesity['UpperBound'] - df_obesity['LowerBound']
df_malnutrition['CI_width'] = df_malnutrition['UpperBound'] - df_malnutrition['LowerBound']

In [23]:
#Create a new column Obesity_level for the obesity data
#Condition : Mean_Estiamte >=30 ----> High
#            Mean_Estimate 25 to 29.9 ----> Moderate
#            Mean_Estimate < 25 ----> Low
df_obesity['Obesity_Level'] = df_obesity['Mean_Estimate'].apply([lambda x:"High" if x >=30 else "Low" if x < 25 else "Moderate"])

In [25]:
#Create a new column Malnutrition level for malnutrition data
#Condition : Mean_Estimate >=20 ----> High
#            Mean_Estimate 10 to 19.9 ----> Modearte
#            Mean_Estimate < 10 ----> Low
df_malnutrition['Malnutrition_Level'] = df_malnutrition['Mean_Estimate'].apply([lambda x:"High" if x >=20 else "Low" if x < 10 else "Moderate"])

In [27]:
spl_cases = {'GLOBAL': 'Global',
             'WB_LMI': 'Low & Middle Income',
             'WB_HI': 'High Income',
             'WB_LI': 'Low Income',
             'EMR': 'Eastern Mediterranean Region',
             'EUR': 'Europe',
             'AFR': 'Africa',
             'SEAR': 'South-East Asia Region',
             'WPR': 'Western Pacific Region',
             'AMR': 'Americas Region',
             'WB_UMI': 'Upper Middle Income'
            }

In [29]:
def country_rename(df):
    """This is a function to rename the country column in order to make it more legible."""
    for i in range(0,len(df)):
        try:
            df.loc[i,'Country'] = pycountry.countries.get(alpha_3=df['Country'][i]).name
        except AttributeError:
            df.loc[i,'Country'] = spl_cases[df['Country'][i]]
    return df

In [31]:
#Rename the Country names by calling the function country_rename(df)
df_obesity = country_rename(df_obesity)
df_malnutrition = country_rename(df_malnutrition)

In [33]:
#Check whether the data has any missing values
df_obesity.info(),df_malnutrition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27720 entries, 0 to 27719
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Region         26268 non-null  object 
 1   Gender         27720 non-null  object 
 2   Year           27720 non-null  int64  
 3   LowerBound     27720 non-null  float64
 4   UpperBound     27720 non-null  float64
 5   Mean_Estimate  27720 non-null  float64
 6   Country        27720 non-null  object 
 7   Age_group      27720 non-null  object 
 8   CI_width       27720 non-null  float64
 9   Obesity_Level  27720 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27720 entries, 0 to 27719
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Region              26268 non-null  object 
 1   Gender              27720 non-null  obje

(None, None)

In [35]:
#Check the region column values(Which has missing values) of obesity data
df_obesity['Region'].value_counts(dropna=False)

Region
Europe                   6864
Africa                   6204
Americas                 4884
Western Pacific          4092
Eastern Mediterranean    2904
None                     1452
South-East Asia          1320
Name: count, dtype: int64

In [37]:
#Check the country names where the region column is empty
df_obesity[df_obesity['Region'].isnull()]['Country'].value_counts()

Country
Western Pacific Region          132
High Income                     132
Low & Middle Income             132
South-East Asia Region          132
Americas Region                 132
Low Income                      132
Global                          132
Eastern Mediterranean Region    132
Upper Middle Income             132
Europe                          132
Africa                          132
Name: count, dtype: int64

In [39]:
#Check the region column values of malnutrition data
df_malnutrition['Region'].value_counts(dropna=False)

Region
Europe                   6864
Africa                   6204
Americas                 4884
Western Pacific          4092
Eastern Mediterranean    2904
None                     1452
South-East Asia          1320
Name: count, dtype: int64

In [41]:
#Check the country names where the region column is empty
df_malnutrition[df_malnutrition['Region'].isnull()]['Country'].value_counts()

Country
South-East Asia Region          132
Low Income                      132
Africa                          132
Americas Region                 132
High Income                     132
Low & Middle Income             132
Eastern Mediterranean Region    132
Europe                          132
Upper Middle Income             132
Western Pacific Region          132
Global                          132
Name: count, dtype: int64

In [43]:
#After the analysing the region and country columns the country name can be assigned to a region as given in the below dictionary
#Key of the dictionary is the country and its corresponding value is the assumed region
fill_region = {"Eastern Mediterranean Region":"Eastern Mediterranean",
              "Western Pacific Region":"Western Pacific",
              "South-East Asia Region":"South-East Asia",
              "Americas Region":"Americas",
              "Europe":"Europe",
              "Africa":"Africa",
              "Upper Middle Income":"Global",
              "High Income":"Global",
              "Low & Middle Income":"Global",
              "Low Income":"Global",
              "Global":"Global"}

In [45]:
#To handle the missing values of obesity data with the fill_region dictionary value
for i in range (0,len(df_obesity)):
    if df_obesity['Region'][i]== None:
        df_obesity.loc[i,'Region'] = fill_region[df_obesity['Country'][i]]

In [47]:
#To handle the missing values of malnutrition data with the fill_region dictionary value
for i in range (0,len(df_malnutrition)):
    if df_malnutrition['Region'][i]== None:
        df_malnutrition.loc[i,'Region'] = fill_region[df_malnutrition['Country'][i]]

In [49]:
#Now the data is having zero NULL Values
df_obesity.info(),df_malnutrition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27720 entries, 0 to 27719
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Region         27720 non-null  object 
 1   Gender         27720 non-null  object 
 2   Year           27720 non-null  int64  
 3   LowerBound     27720 non-null  float64
 4   UpperBound     27720 non-null  float64
 5   Mean_Estimate  27720 non-null  float64
 6   Country        27720 non-null  object 
 7   Age_group      27720 non-null  object 
 8   CI_width       27720 non-null  float64
 9   Obesity_Level  27720 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27720 entries, 0 to 27719
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Region              27720 non-null  object 
 1   Gender              27720 non-null  obje

(None, None)

# ## Create SQLAlchemy engine to create the data base
username = "root"
password = "Pgnkka#"
host = "localhost"
port = 3306
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}")
create_db_query = """
                     CREATE DATABASE obesity_malnutrition;
                     """
with engine.connect() as conn:
    conn.execute(text(create_db_query))
    conn.commit()

In [51]:
## Insert the df to mysql
username = "root"
password = "Pgnkka#"
host = "localhost"
port = 3306
database = "obesity_malnutrition"
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [53]:
#Insert the two dataframes to SQL
df_obesity.to_sql("obesity",engine,index=False)
df_malnutrition.to_sql("malnutrition",engine,index=False)

27720

drp_q = """DROP TABLE obesity;"""
drp_q1 = """DROP TABLE malnutrition"""
with engine.connect() as conn:
    conn.execute(text(drp_q))
    conn.execute(text(drp_q1))
    conn.commit()